# Filename parsing

## Introduction

piece of code that delves into the intricacies of space weather monitoring using data from the China Seismo-Electromagnetic Satellite (CSES). 

## Project Setup

The project directory, named CSES_files, serves as the repository for our data files. These files, stored in `HDF5` format, contain valuable measurements from various instruments aboard the CSES satellite. The initial step involves importing the necessary libraries:

In [2]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import h5py
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import xarray as xr
import xarray
from shapely import geometry
from glob import glob
from shapely.geometry import Point, box

Each library plays a crucial role in data manipulation, visualization, and geographic data handling. For instance, `xarray` is used for handling **multi-dimensional arrays** efficiently, while `geopandas` provides tools for **geographic data manipulation**.

## File Paths and Dataset Handling

### define project directory and the names for different datasets

The code begins by defining the project directory and listing the `HDF5` files to be processed:

In [3]:
project_dir = "/home/wvuser/CSESfiles"

print(f"Percorso cartella di progetto: {project_dir}")

EFD1 = 'CSES_01_EFD_1_L02_A1_213330_20211206_164953_20211206_172707_000.h5'
HEP1 = 'CSES_01_HEP_1_L02_A4_176401_20210407_182209_20210407_190029_000.h5'
HEP4 = 'CSES_01_HEP_4_L02_A4_202091_20210923_184621_20210923_192441_000.h5'
LAP1 = 'CSES_01_LAP_1_L02_A3_174201_20210324_070216_20210324_073942_000.h5'
SCM1 = 'CSES_01_SCM_1_L02_A2_183380_20210523_154551_20210523_162126_000.h5'
HEPD = 'CSES_HEP_DDD_0219741_20220117_214156_20220117_230638_L3_0000267631.h5'


Percorso cartella di progetto: /home/wvuser/CSESfiles


The `dataset` function is then defined to open an `xarray` dataset from a given file path:

In [4]:
# Function to open an xarray dataset from a given path
def dataset(path):
    return xarray.open_dataset(path, engine = 'h5netcdf', phony_dims = 'sort')

# Function to list all variable names in a dataset
def variables(data):
    return list(data.keys())

### List of file paths to be processed

In [5]:

file_list = [
    os.path.join(project_dir, EFD1),
    os.path.join(project_dir, HEP1),
    os.path.join(project_dir, HEP4),
    os.path.join(project_dir, LAP1),
    os.path.join(project_dir, SCM1),
    os.path.join(project_dir, HEPD)
]

print(file_list[0])

# Redefine the dataset function to open xarray datasets
def dataset(path):
    return xarray.open_dataset(path, engine = 'h5netcdf', phony_dims = 'sort')

# Redefine the variables function to list all variable names in a dataset
def variables(data):
    return list(data.keys())



/home/wvuser/CSESfiles/CSES_01_EFD_1_L02_A1_213330_20211206_164953_20211206_172707_000.h5


## Extracting Metadata

To understand the data better, the code extracts `metadata` such as `start` and `end dates`, and `orbit numbers` from the filenames. This is achieved using the `extract_dates` and `extract_orbit` functions:

### Extract the name from a file name

In [6]:
def get_file_name(file_path):
    return os.path.basename(file_path)

### Extract satellite number from a file name

In [7]:
def extract_satellite_number(file_name):
    try:
        parts = file_name.split('_')
        satellite_number = parts[1]
        return satellite_number
    except IndexError:
        print(f"Errore nell'estrazione del numero del satellite per il file {file_name}")
        return None

### Extract instrument code from a file name

In [8]:
def extract_instrument_code(file_name):
    try:
        parts = file_name.split('_')
        instrument_code = parts[2]
        return instrument_code
    except IndexError:
        print(f"Errore nell'estrazione del codice strumento per il file {file_name}")
        return None

### Extract instrument number from a file name

In [9]:
def extract_instrument_number(file_name):
    try:
        parts = file_name.split('_')
        instrument_number = parts[3]
        return instrument_number
    except IndexError:
        print(f"Errore nell'estrazione del numero strumento per il file {file_name}")
        return None

### Extract data level from a file name

In [10]:
def extract_data_level(file_name):
    try:
        parts = file_name.split('_')
        data_level = parts[4]
        return data_level
    except IndexError:
        print(f"Errore nell'estrazione del livello dei dati per il file {file_name}")
        return None

### Extract orbit number from a file name

In [11]:
def extract_orbit(file_name):
    try:
        base_name = os.path.basename(file_name)
        parts = base_name.split('_')
        start_index = None
        for i in range(len(parts)):
            if parts[i].isdigit() and len(parts[i]) == 8: 
                start_index = i
                break
    
        if start_index is None:
            raise ValueError(f"Formato data non trovato nel nome del file: {file_name}")
        
        orbit = parts[start_index - 1]  
        return orbit
    except ValueError as e:
        print(f"Errore nel parsing dell'orbita per il file {file_name}: {e}")
        return None

### Extract start and end dates from a file name

In [12]:

def extract_dates(file_name):
    try:
        base_name = os.path.basename(file_name) #returns the final component of a pathname
        parts = base_name.split('_')
        
        #find the index of the part that contains the start_date
        start_index = None
        for i in range(len(parts)):
            if parts[i].isdigit() and len(parts[i]) == 8:  # find the part with data format YYYYMMDD
                start_index = i
                break
        
        if start_index is None:
            raise ValueError(f"Formato data non trovato nel nome del file: {file_name}")
        
        start_date_str = '_'.join(parts[start_index:start_index + 2]) 
        end_date_str = '_'.join(parts[start_index + 2:start_index + 4])  
        
        start_date = datetime.strptime(start_date_str, '%Y%m%d_%H%M%S')
        end_date = datetime.strptime(end_date_str, '%Y%m%d_%H%M%S')
        
        return start_date, end_date
    except ValueError as e:
        print(f"Errore nel parsing delle date per il file {file_name}: {e}")
        return None, None

### parse_filename function to include the data


In [13]:
def parse_filename(file_name):
    file = get_file_name(file_name)
    satellite_nr = extract_satellite_number(file_name)
    instrument_code = extract_instrument_code(file_name)
    instrument_nr = extract_instrument_number(file_name)
    data_l = extract_data_level(file_name)
    start_date, end_date = extract_dates(file_name)
    semiorbit_nr = extract_orbit(file_name)
    return {
        'file_name': file,
        "satellite_nr": satellite_nr,
        "instrument_code": instrument_code,
        "instrument_nr": instrument_nr,
        "data_l":data_l,
        "semiorbit_nr": semiorbit_nr,
        "start_date": start_date, 
        "end_date" : end_date
    }

## Creating the DataFrame

The parsed metadata is stored in a list of dictionaries, which is then converted into a pandas DataFrame:

In [14]:
data = []

for file in file_list:
    metadata = parse_filename(file)
    if metadata:
        data.append(metadata)
    #metadata["semiorbit_nr"]
    #semiorbits_geo[metadata["semiorbit_nr"]]
    # {
    #     "start_date": ...
    #     "start_date": ...
    #     "start_date": ...
    # }
if data:
    columns = list(data[0].keys())
else:
    columns = []


df = pd.DataFrame(data, columns=columns)

df

,file_name,satellite_nr,instrument_code,instrument_nr,data_l,semiorbit_nr,start_date,end_date
0,CSES_01_EFD_1_L02_A1_213330_20211206_164953_20...,01,EFD,1,L02,213330,2021-12-06 16:49:53,2021-12-06 17:27:07
1,CSES_01_HEP_1_L02_A4_176401_20210407_182209_20...,01,HEP,1,L02,176401,2021-04-07 18:22:09,2021-04-07 19:00:29
2,CSES_01_HEP_4_L02_A4_202091_20210923_184621_20...,01,HEP,4,L02,202091,2021-09-23 18:46:21,2021-09-23 19:24:41
3,CSES_01_LAP_1_L02_A3_174201_20210324_070216_20...,01,LAP,1,L02,174201,2021-03-24 07:02:16,2021-03-24 07:39:42
4,CSES_01_SCM_1_L02_A2_183380_20210523_154551_20...,01,SCM,1,L02,183380,2021-05-23 15:45:51,2021-05-23 16:21:26
5,CSES_HEP_DDD_0219741_20220117_214156_20220117_...,HEP,DDD,0219741,20220117,0219741,2022-01-17 21:41:56,2022-01-17 23:06:38


## Geographic Data Handling

The `polygon` function allows us to create a polygon from geographic coordinates and filter out data points that fall outside this polygon:

In [15]:
def polygon(points, data):
    
    ds = dataset(data)

    geo_lat = ds.GEO_LAT
    geo_lon = ds.GEO_LON


    latitudes = [point[1] for point in points]
    longitudes = [point[0] for point in points]

    lat_min = min(latitudes)
    lat_max = max(latitudes)
    lon_min = min(longitudes)
    lon_max = max(longitudes)

    lat_mask = (geo_lat >= lat_min) & (geo_lat <= lat_max)
    lon_mask = (geo_lon >= lon_min) & (geo_lon <= lon_max)

    print(f"Bounding Box - lat_min: {lat_min}, lat_max: {lat_max}, lon_min: {lon_min}, lon_max: {lon_max}")

    final_mask = lat_mask + lon_mask

    filtered_subset = ds.where(final_mask, drop=True)

    if filtered_subset.GEO_LAT.size > 0 and filtered_subset.GEO_LON.size > 0:
        return(filtered_subset)

In [16]:
polygon_points = [(100.0, 30.0), (120.0, 30.0), (120.0, 50.0), (100.0, 50.0)]
print(polygon_points[0][0])

#Test the polygon function with a polygon and one of the data files
filtered_points = polygon(polygon_points, file_list[0])

print(filtered_points.GEO_LAT.values)

100.0
Bounding Box - lat_min: 30.0, lat_max: 50.0, lon_min: 100.0, lon_max: 120.0
[[49.94165 ]
 [49.81457 ]
 [49.687496]
 [49.56037 ]
 [49.43323 ]
 [49.3061  ]
 [49.17893 ]
 [49.051746]
 [48.924522]
 [48.79731 ]
 [48.67008 ]
 [48.542824]
 [48.415565]
 [48.288284]
 [48.160988]
 [48.033672]
 [47.906357]
 [47.779003]
 [47.65163 ]
 [47.524273]
 [47.396866]
 [47.269478]
 [47.14204 ]
 [47.014606]
 [46.887154]
 [46.759686]
 [46.6322  ]
 [46.504692]
 [46.377213]
 [46.24968 ]
 [46.122166]
 [45.99462 ]
 [45.867058]
 [45.73951 ]
 [45.611927]
 [45.484356]
 [45.356743]
 [45.229145]
 [45.101517]
 [44.973896]
 [44.846245]
 [44.71858 ]
 [44.5909  ]
 [44.4632  ]
 [44.335487]
 [44.207733]
 [44.08002 ]
 [43.952267]
 [43.824516]
 [43.696747]
 [43.568966]
 [43.44119 ]
 [43.313374]
 [43.18557 ]
 [43.057724]
 [42.929905]
 [42.802044]
 [42.67419 ]
 [42.546318]
 [42.418415]
 [42.290527]
 [42.1626  ]
 [42.03468 ]
 [41.906742]
 [41.778805]
 [41.650864]
 [41.522903]
 [41.394947]
 [41.26696 ]
 [41.138973]
 [41.010

The following provides functionalities to find files by orbit number in a DataFrame, extract geographical coordinates from HDF5 files, and filter these coordinates within a specific bounding box.



### Finding a File by Orbit Number

In [17]:
def find_file_by_orbit(df, orbit_nr):
    return df[df['orbit_nr'] == orbit_nr]

- **purpose**: This function filters a DataFrame to find rows that match a specified orbit number.-
- **Parameters**:
    - *df* (pandas.DataFrame): The DataFrame containing orbit data.
    - *orbit_nr* (int): The orbit number to filter by.
- **Returns**: A DataFrame containing only the rows where the orbit_nr column matches the specified orbit number.

### Extracting Coordinates from HDF5 File

In [18]:
# We will use th h5py package for read the file and estract the coordinates
def extract_coordinates(file_name):
    with h5py.File(file_name, 'r') as f:
        latitudes = f['GEO_LAT'][:]
        longitudes = f['GEO_LON'][:]
        coordinates = np.column_stack((longitudes, latitudes))
    return coordinates

- **purpose**: This function reads geographical coordinates (latitude and longitude) from an HDF5 file and returns them as a NumPy array.
- **Parameters**:
    - *file_name* (str): The name of the HDF5 file to read.
- **Returns**: A NumPy array of shape (N, 2) where N is the number of coordinate pairs, containing longitude and latitude pairs.

### Filtering Coordinates Within a Bounding Box

In [19]:
# filtering the coordinates
def filter_coordinates_in_bbox(coords, bbox):
    points_in_box = [coord for coord in coords if bbox.contains(Point(coord))]
    for coord in points_in_box:
        print(coord)
    return points_in_box

- **Purpose**: This function filters a list of coordinates to only include those that lie within a specified bounding box.
- **Parameters**:
    - *coords* (list or numpy.ndarray): A list or array of coordinate pairs (longitude, latitude).
    - *bbox* (shapely.geometry.box): A bounding box to filter coordinates.
- **Returns**: A list of coordinates that are within the bounding box. Each coordinate is a tuple (longitude, latitude).

In [20]:
bbox = box(-55.85, -69.4844, -106.0156, 69.5816 )

orbit_coords = extract_coordinates(file_list[3])
print(orbit_coords)

filtered_coords = filter_coordinates_in_bbox(orbit_coords, bbox)

[[ -55.85      -69.4844  ]
 [ -56.059998  -69.3089  ]
 [ -56.266     -69.1331  ]
 ...
 [-106.0156     69.5816  ]
 [-106.2301     69.7571  ]
 [-106.4406     69.9265  ]]
[-56.059998 -69.3089  ]
[-56.266  -69.1331]
[-56.469  -68.9572]
[-56.669 -68.781]
[-56.865997 -68.6046  ]
[-57.059998 -68.428   ]
[-57.251  -68.2513]
[-57.440002 -68.0743  ]
[-57.626  -67.8971]
[-57.809  -67.7197]
[-57.989998 -67.5422  ]
[-58.168  -67.3645]
[-58.344  -67.1866]
[-58.517  -67.0086]
[-58.688004 -66.8303  ]
[-58.857002 -66.6519  ]
[-59.023003 -66.4734  ]
[-59.188004 -66.2947  ]
[-59.35   -66.1159]
[-59.510002 -65.9369  ]
[-59.668  -65.7577]
[-59.823997 -65.5785  ]
[-59.977997 -65.3991  ]
[-60.129997 -65.2195  ]
[-60.281  -65.0398]
[-60.429  -64.8599]
[-60.575996 -64.6799  ]
[-60.72   -64.4999]
[-60.863  -64.3196]
[-61.004997 -64.1393  ]
[-61.143997 -63.9589  ]
[-61.281998 -63.7783  ]
[-61.419  -63.5977]
[-61.554  -63.4169]
[-61.686996 -63.236   ]
[-61.819 -63.055]
[-61.949997 -62.8739  ]
[-62.079002 -62.6928